In [4]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [7]:
AS= pd.read_csv('../notebook/data/Alaska_Airlines.csv',skiprows=7)

FileNotFoundError: [Errno 2] File b'../notebook/data/Alaska_Airlines.csv' does not exist: b'../notebook/data/Alaska_Airlines.csv'

In [3]:
df= pd.DataFrame(AS)
print(len(df))

8384


In [4]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
39,AS,01/03/2018,18.0,N419AS,SEA,06:00,00:00
40,AS,01/03/2018,480.0,N251AK,PDX,05:25,00:00
58,AS,01/04/2018,18.0,N459AS,SEA,06:00,00:00
59,AS,01/04/2018,480.0,N273AK,PDX,05:25,00:00
107,AS,01/06/2019,1406.0,N634VA,LAX,19:12,00:00
...,...,...,...,...,...,...,...
7342,AS,11/03/2018,1406.0,N631VA,LAX,19:11,00:00
7532,AS,11/14/2018,1406.0,N632VA,LAX,19:12,00:00
7550,AS,11/15/2018,1406.0,N362VA,LAX,19:12,00:00
7749,AS,11/26/2018,1420.0,N921VA,LAX,07:58,00:00


In [5]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AS,01/01/2016,18.0,N481AS,SEA,06:00,06:01
1,AS,01/01/2017,18.0,N474AS,SEA,05:55,06:54
2,AS,01/01/2018,18.0,N282AK,SEA,06:00,06:01
3,AS,01/01/2018,480.0,N557AS,PDX,05:25,05:30
4,AS,01/01/2019,18.0,N273AK,SEA,06:04,05:55


In [6]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [7]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AS,01/01/2016,18.0,N481AS,SEA,06:00,06:01
1,AS,01/01/2017,18.0,N474AS,SEA,05:55,06:54
2,AS,01/01/2018,18.0,N282AK,SEA,06:00,06:01
3,AS,01/01/2018,480.0,N557AS,PDX,05:25,05:30
4,AS,01/01/2019,18.0,N273AK,SEA,06:04,05:55


In [8]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AS,2016-01-01,18.0,N481AS,SEA,06:00,06:01
1,AS,2017-01-01,18.0,N474AS,SEA,05:55,06:54
2,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01
3,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30
4,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55
...,...,...,...,...,...,...,...
8378,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11
8379,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58
8380,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31
8381,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15


In [9]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,AS,2016-01-01,18.0,N481AS,SEA,06:00,06:01,2016-01-01
1,AS,2017-01-01,18.0,N474AS,SEA,05:55,06:54,2017-01-01
2,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01
3,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01
4,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01
...,...,...,...,...,...,...,...,...
8378,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31
8379,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31
8380,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31
8381,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31


In [10]:
df.loc[df['Actual Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
1908,AS,2018-04-10,1416.0,N835VA,LAX,00:23,24:00,2018-04-10
2383,AS,2019-04-26,104.0,N264AK,SFO,23:55,24:00,2019-04-26
3037,AS,2019-05-18,104.0,N287AK,SFO,23:55,24:00,2019-05-18
4198,AS,2018-06-26,1416.0,N639VA,LAX,00:37,24:00,2018-06-26
5376,AS,2019-08-03,468.0,N453AS,LAX,23:55,24:00,2019-08-03
8042,AS,2018-12-13,1026.0,N625VA,SFO,23:44,24:00,2018-12-13


In [11]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')

In [12]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
39,AS,2018-01-03,18.0,N419AS,SEA,06:00,00:00,2018-01-03
40,AS,2018-01-03,480.0,N251AK,PDX,05:25,00:00,2018-01-03
58,AS,2018-01-04,18.0,N459AS,SEA,06:00,00:00,2018-01-04
59,AS,2018-01-04,480.0,N273AK,PDX,05:25,00:00,2018-01-04
107,AS,2019-01-06,1406.0,N634VA,LAX,19:12,00:00,2019-01-06
...,...,...,...,...,...,...,...,...
7532,AS,2018-11-14,1406.0,N632VA,LAX,19:12,00:00,2018-11-14
7550,AS,2018-11-15,1406.0,N362VA,LAX,19:12,00:00,2018-11-15
7749,AS,2018-11-26,1420.0,N921VA,LAX,07:58,00:00,2018-11-26
7813,AS,2018-11-30,1024.0,N925VA,SFO,22:23,00:00,2018-11-30


In [13]:
df.loc[df['Actual Arrival Time']=='00:00','Actual Arrival Date']+=dt.timedelta(days=1)

In [14]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [15]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [16]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [17]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,AS,2016-01-01,18.0,N481AS,SEA,06:00,06:01,2016-01-01,2016-01-01 06:01,2016-01-01 06:00
1,AS,2017-01-01,18.0,N474AS,SEA,05:55,06:54,2017-01-01,2017-01-01 06:54,2017-01-01 05:55
2,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01,2018-01-01 06:01,2018-01-01 06:00
3,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01,2018-01-01 05:30,2018-01-01 05:25
4,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01,2019-01-01 05:55,2019-01-01 06:04
...,...,...,...,...,...,...,...,...,...,...
8378,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31,2018-12-31 15:11,2018-12-31 15:30
8379,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31,2018-12-31 18:58,2018-12-31 19:12
8380,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31,2018-12-31 21:31,2018-12-31 22:12
8381,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31,2018-12-31 23:15,2018-12-31 23:57


In [18]:
FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,AS,2016-01-01,18.0,N481AS,SEA,06:00,06:01,2016-01-01,2016-01-01 06:01:00,2016-01-01 06:00:00
1,AS,2017-01-01,18.0,N474AS,SEA,05:55,06:54,2017-01-01,2017-01-01 06:54:00,2017-01-01 05:55:00
2,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01,2018-01-01 06:01:00,2018-01-01 06:00:00
3,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01,2018-01-01 05:30:00,2018-01-01 05:25:00
4,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01,2019-01-01 05:55:00,2019-01-01 06:04:00
...,...,...,...,...,...,...,...,...,...,...
8378,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31,2018-12-31 15:11:00,2018-12-31 15:30:00
8379,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31,2018-12-31 18:58:00,2018-12-31 19:12:00
8380,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31,2018-12-31 21:31:00,2018-12-31 22:12:00
8381,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31,2018-12-31 23:15:00,2018-12-31 23:57:00


In [19]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,AS,2016-01-01,18.0,N481AS,SEA,06:00,06:01,2016-01-01,2016-01-01 06:01:00,2016-01-01 06:00:00,-1.0
1,AS,2017-01-01,18.0,N474AS,SEA,05:55,06:54,2017-01-01,2017-01-01 06:54:00,2017-01-01 05:55:00,-59.0
2,AS,2018-01-01,18.0,N282AK,SEA,06:00,06:01,2018-01-01,2018-01-01 06:01:00,2018-01-01 06:00:00,-1.0
3,AS,2018-01-01,480.0,N557AS,PDX,05:25,05:30,2018-01-01,2018-01-01 05:30:00,2018-01-01 05:25:00,-5.0
4,AS,2019-01-01,18.0,N273AK,SEA,06:04,05:55,2019-01-01,2019-01-01 05:55:00,2019-01-01 06:04:00,9.0
...,...,...,...,...,...,...,...,...,...,...,...
8378,AS,2018-12-31,1036.0,N640VA,SJC,15:30,15:11,2018-12-31,2018-12-31 15:11:00,2018-12-31 15:30:00,19.0
8379,AS,2018-12-31,1406.0,N923VA,LAX,19:12,18:58,2018-12-31,2018-12-31 18:58:00,2018-12-31 19:12:00,14.0
8380,AS,2018-12-31,1412.0,N839VA,LAX,22:12,21:31,2018-12-31,2018-12-31 21:31:00,2018-12-31 22:12:00,41.0
8381,AS,2018-12-31,1416.0,N851VA,LAX,23:57,23:15,2018-12-31,2018-12-31 23:15:00,2018-12-31 23:57:00,42.0


In [20]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2019
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)

Total Alaska Airlines flights that landed at JFK in 2019: 3407
Flights that arrived early in 2019: 2114
Flights that arrived more than 20 mins early in 2019: 821


In [21]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2018
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)

Total Alaska Airlines flights that landed at JFK in 2018: 4082
Flights that arrived early in 2018: 2458
Flights that arrived more than 20 mins early in 2018: 993


In [22]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2017
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)

Total Alaska Airlines flights that landed at JFK in 2017: 421
Flights that arrived early in 2017: 290
Flights that arrived more than 20 mins early in 2017: 103


In [23]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2016
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)

Total Alaska Airlines flights that landed at JFK in 2016: 366
Flights that arrived early in 2016: 276
Flights that arrived more than 20 mins early in 2016: 127


In [24]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2015
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

Total Alaska Airlines flights that landed at JFK in 2015: 107
Flights that arrived early in 2015: 84
Flights that arrived more than 20 mins early in 2015: 56


In [25]:
#df.loc[df['Actual Arrival Time']=='00:00']

In [26]:


# dictionary of lists  
 
year =["2015","2016","2017","2018","2019"]
total_flights= [Total_15, Total_16, Total_17, Total_18, Total_19]
early_arrival=[Early_Arrivals_15,Early_Arrivals_16,Early_Arrivals_17,Early_Arrivals_18,Early_Arrivals_19]
inflated_padding=[Twenty_Mins_Early_15,Twenty_Mins_Early_16,Twenty_Mins_Early_17,Twenty_Mins_Early_18,Twenty_Mins_Early_19]
Carrier_Code=df['Carrier Code'][:5]
dict = {'Carrier Code': Carrier_Code,'year': year, 'Total flights': total_flights, 'Early Arrivals': early_arrival, 'Inflated padding(>20mins early)': inflated_padding} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'../output/Alaska_Airlines.csv',index=False) 